In [1]:
import os
import pandas as pd
from IPython.display import display
from transformers import AutoTokenizer, AutoModelForTokenClassification
from torch import nn
import torch

# Spécifiez le chemin du modèle DistilBERT
distilbert_model_path = r'C:\Users\vikne\Documents\Master 2\Semestre 9\Intelligence artificielle\Travel-Order-Resolver\ai\nlp\models\tokens_classification\camembert'

# Charger le tokenizer et le modèle DistilBERT
try:
    tokenizer = AutoTokenizer.from_pretrained(distilbert_model_path)
    model = AutoModelForTokenClassification.from_pretrained(distilbert_model_path)
    print("Modèle DistilBERT et tokenizer chargés avec succès.")
except Exception as e:
    print(f"Erreur lors du chargement du modèle DistilBERT ou du tokenizer: {e}")

# Liste des phrases pour l'évaluation
phrases = [
    "Je veux partir de Strasbourg et arriver à Lille.",
    "Je veux partir de Bordeaux et arriver à Arcachon.",
    "Je veux partir de Dijon et arriver à Clermont-Ferrand.",
    "Je veux partir de Poitiers et arriver à Limoges.",
    "Je veux partir de Rouen et arriver à Évreux.",
    "Je veux partir de Chambéry et arriver à Grenoble.",
    "Je veux partir de Brest et arriver à Quimper.",
    "Je veux partir de Laval et arriver à Angers.",
    "Je veux partir de Perpignan et arriver à Narbonne.",
    "Je veux partir de Annecy et arriver à Genève.",
    "Je veux partir de Lyon et arriver à Valence.",
    "Je veux partir de Rennes et arriver à Saint-Malo.",
    "Je veux partir de Marseille et arriver à Toulon.",
    "Je veux partir de Besançon et arriver à Belfort.",
    "Je veux partir de Genève et arriver à Lausanne."
]

# Liste pour stocker les résultats
results = []

# Fonction pour afficher les prédictions avec DistilBERT
for phrase in phrases:
    try:
        # Tokenizer la phrase
        inputs = tokenizer(phrase, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            # Passer les tokens dans le modèle
            outputs = model(**inputs)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=2)

        # Obtenir les étiquettes prédits pour chaque token
        tokens = tokenizer.tokenize(tokenizer.decode(inputs['input_ids'][0]))  # Récupérer les tokens
        labels = [model.config.id2label[pred.item()] for pred in predictions[0]]

        # Ajouter les entités détectées à la liste des résultats
        for token, label in zip(tokens, labels):
            if label != 'O':  # On ignore les tokens avec le label 'O'
                results.append({"Phrase": phrase, "Entity": token, "Label": label})

    except Exception as e:
        print(f"Erreur lors du traitement de la phrase '{phrase}': {e}")

# Convertir les résultats en DataFrame
df = pd.DataFrame(results)

# Afficher le tableau
if not df.empty:
    display(df)
else:
    print("Aucune entité détectée.")

,Phrase,Entity,Label
0,Je veux aller de Strasbourg à Lille.,strasbourg,B-DEP
1,Je veux aller de Strasbourg à Lille.,lille,B-ARR
2,Je veux aller de Bordeaux à Arcachon.,bordeaux,B-DEP
3,Je veux aller de Bordeaux à Arcachon.,arcachon,B-ARR
4,Je veux aller de Dijon à Clermont-Ferrand.,dijon,B-DEP
5,Je veux aller de Dijon à Clermont-Ferrand.,clermont-ferrand,B-ARR
6,Je veux aller de Poitiers à Limoges.,poitiers,B-DEP
7,Je veux aller de Poitiers à Limoges.,limoges,B-ARR
8,Je veux aller de Rouen à Évreux.,rouen,B-DEP
9,Je veux aller de Rouen à Évreux.,évreux,B-ARR
